In [1]:
import numpy as np
import os
import glob

In [2]:
t_start = 16650
t_end = 17000
dt = 0.1
seq_dir = "02/"
save_dir = "02_processed/"

voxel_resolution = 0.2
grid_lengths = np.array([40.0, 40.0, 4.0])
NUM_CLASSES = 25

grid_dim = 2 * grid_lengths / voxel_resolution
grid_dim = [int(grid_dim[0]), int(grid_dim[1]), int(grid_dim[2]), NUM_CLASSES] # 0 is free
print(grid_dim)

x = np.arange(-grid_lengths[0], grid_lengths[0], voxel_resolution) + voxel_resolution/2
y = np.arange(-grid_lengths[1], grid_lengths[1], voxel_resolution) + voxel_resolution/2
z = np.arange(-grid_lengths[2], grid_lengths[2], voxel_resolution) + voxel_resolution/2
xv, yv, zv = np.meshgrid(x, y, z)

[400, 400, 40, 25]


In [3]:
if not os.path.exists(save_dir):
    os.mkdir(save_dir)

sensors = glob.glob(seq_dir + "velodyne*")
sensors = sorted([int(sensor.split("velodyne")[1]) for sensor in sensors])
ego_sensor = sensors[1]

In [4]:
times = []

t_from = open(seq_dir + "times0.txt", "r")
for t_stamp in t_from.readlines():
    t_frame = t_stamp.split(", ")[0]
    if int(t_frame) < t_start:
        continue
    if int(t_frame) >= t_end:
        continue
    t_frame = (float(t_frame) - t_start) * dt
    times.append(t_frame)
t_from.close()

times = np.array(times)
print(times)
np.savetxt(save_dir + 'times.txt', times)

[ 0.   0.1  0.2  0.3  0.4  0.5  0.6  0.7  0.8  0.9  1.   1.1  1.2  1.3
  1.4  1.5  1.6  1.7  1.8  1.9  2.   2.1  2.2  2.3  2.4  2.5  2.6  2.7
  2.8  2.9  3.   3.1  3.2  3.3  3.4  3.5  3.6  3.7  3.8  3.9  4.   4.1
  4.2  4.3  4.4  4.5  4.6  4.7  4.8  4.9  5.   5.1  5.2  5.3  5.4  5.5
  5.6  5.7  5.8  5.9  6.   6.1  6.2  6.3  6.4  6.5  6.6  6.7  6.8  6.9
  7.   7.1  7.2  7.3  7.4  7.5  7.6  7.7  7.8  7.9  8.   8.1  8.2  8.3
  8.4  8.5  8.6  8.7  8.8  8.9  9.   9.1  9.2  9.3  9.4  9.5  9.6  9.7
  9.8  9.9 10.  10.1 10.2 10.3 10.4 10.5 10.6 10.7 10.8 10.9 11.  11.1
 11.2 11.3 11.4 11.5 11.6 11.7 11.8 11.9 12.  12.1 12.2 12.3 12.4 12.5
 12.6 12.7 12.8 12.9 13.  13.1 13.2 13.3 13.4 13.5 13.6 13.7 13.8 13.9
 14.  14.1 14.2 14.3 14.4 14.5 14.6 14.7 14.8 14.9 15.  15.1 15.2 15.3
 15.4 15.5 15.6 15.7 15.8 15.9 16.  16.1 16.2 16.3 16.4 16.5 16.6 16.7
 16.8 16.9 17.  17.1 17.2 17.3 17.4 17.5 17.6 17.7 17.8 17.9 18.  18.1
 18.2 18.3 18.4 18.5 18.6 18.7 18.8 18.9 19.  19.1 19.2 19.3 19.4 19.5
 19.6 

In [5]:
def get_frame_str(t_frame, t_start, t_end):
    if int(t_frame) < t_start:
        return None
    if int(t_frame) >= t_end:
        return None
    t_frame = int(t_frame) - t_start
    return str(t_frame).zfill(6)

In [6]:
# Original view
poses = {}
sorted_poses_all = {}
inv_first = None 
for sensor in sensors:
    poses[sensor] = {}
    # Get poses
    for pose_file in os.listdir(seq_dir + "pose" + str(sensor)):
        t_frame = pose_file.split(".")[0]
        frame_str = get_frame_str(t_frame, t_start, t_end)
        if frame_str:
            pose = np.load(seq_dir + "pose" + str(sensor) + "/" + pose_file)
            poses[sensor][frame_str] = pose

    # Sort poses
    sorted_poses = [poses[sensor][fr] for fr in sorted(poses[sensor].keys())]
    # Make first pose origin
    if sensor == 0:
        inv_first = np.linalg.inv(sorted_poses[0])
    for i in range(len(sorted_poses)):
        sorted_poses[i] = np.matmul(inv_first, sorted_poses[i])
        sorted_poses[i] = sorted_poses[i].reshape(-1)[:12]

    sorted_poses = np.array(sorted_poses)
    if sensor == 0:
        np.savetxt(save_dir + '/poses.txt', sorted_poses)
    sorted_poses_all[sensor] = sorted_poses

In [7]:
# Labels for ego sensor
def save_labels(view_num, query):
    to_folder = save_dir + "labels"
    if query:
        to_folder += "_query"
    to_folder += "/"
    
    if not os.path.exists(to_folder):
        os.mkdir(to_folder)
    
    for label_file in os.listdir(seq_dir + "labels" + str(view_num)):
        t_frame = label_file.split(".")[0]
        frame_str = get_frame_str(t_frame, t_start, t_end)
        if frame_str:
            label = np.load(seq_dir + "labels" + str(view_num) + "/" + label_file)
            label.astype('uint32').tofile(to_folder + frame_str + ".label")
            values, counts = np.unique(label, return_counts=True)
            print(values, counts)
        
save_labels(0, False)

[ 1  3  4  5  6  7  8  9 10 12 14 19 20] [ 6562    76    87   262   322  3919 21419   758   430    15   149   104
    61]
[ 1  4  5  6  7  8  9 10 12 14 19 20] [24893   657   230   129  2551 11739  2531   687     3   173   433     6]
[ 1  4  5  6  7  8  9 10 12 14 19 20 22] [23090   342   245   343  4524 11581  2125   499    10   139   343    15
     6]
[ 1  4  5  6  7  8  9 10 12 14 19 20] [25330   419   385   583  8102  6504  1047   528    14    87    31     5]
[ 1  4  5  6  7  8  9 10 12 19 20 22] [ 9732   231   476   927 14622  7537  1258  2413   104  1045    23    78]
[ 1  3  4  5  6  7  8  9 10 12 14 19 20 22] [ 7567   193    64   474   447  5654 18779   622   933    30   115   304
   164     2]
[ 1  4  5  6  7  8  9 10 12 14 19 20] [25280   780   261    87  2268 11722  2680   664     6   160   458     7]
[ 1  4  5  6  7  8  9 10 12 14 19 20] [25038   267   188   196  3411 11278  2195   600     8   120   247    17]
[ 1  4  5  6  7  8  9 10 12 14 19 20] [15065   245   394   868 10

[ 1  4  5  6  7  8  9 10 12 14 19 20] [12802   619   356  1601 15258  7000  1589  1214    19    89    93    24]
[ 1  3  4  5  6  7  8  9 10 12 19 20] [25149   313     3   555   396  8066  6383   417  1145    23   136   111]
[ 1  4  5  6  7  8  9 10 12 14 19 20] [11186   533   238  1908 16443  6404  1654  1094    20    61   157    23]
[ 1  4  5  6  7  8  9 10 12 14 19 20] [13463   427   386  1217 14831  7196  1611  1170    23    94   106    30]
[ 1  4  5  6  7  8  9 10 12 14 19 20] [26083   399   328   554  7874  6166   982   512    15    68    32     3]
[ 1  4  5  6  7  8  9 10 12 14 19 20] [25156   275   214   216  3452 11073  2215   603    10   144   230    18]
[ 1  4  5  6  7  8  9 10 12 14 19 20] [23902   863   215   102  2871 12022  2758   810     4   212   512     6]
[ 1  4  5  6  7  8  9 10 12 14 19 20] [25402   281   223   207  3525 10663  2225   601     7   154   235    18]
[ 1  3  4  5  6  7  8  9 10 12 19 22] [ 9365    44    13   415   251  4998 19207  1557  1217    28   983

In [8]:
# Labels
def save_points(view_num, query):
    to_folder_points = save_dir + "velodyne"
    to_folder_flow = save_dir + "predictions"
    if query:
        to_folder_points += "_query"
        to_folder_flow += "_query"
    to_folder_points += "/"
    to_folder_flow += "/"
    
    if not os.path.exists(to_folder_points):
        os.mkdir(to_folder_points)
    if not os.path.exists(to_folder_flow):
        os.mkdir(to_folder_flow)
    
    for point_file in os.listdir(seq_dir + "velodyne" + str(view_num)):
        t_frame = point_file.split(".")[0]
        frame_str = get_frame_str(t_frame, t_start, t_end)
        if frame_str:
            points = np.load(seq_dir + "velodyne" + str(view_num) + "/" + point_file) # Point cloud
            instances = np.load(seq_dir + "instances" + str(view_num) + "/" + point_file) # Instances per point
            velocities = np.load(seq_dir + "velocities" + str(view_num) + "/" + point_file) # Per-instance velocity
            flow = np.zeros(points.shape, dtype=np.float32) # Actual movement of things
            for row in velocities:
                ind = int(row[0])
                velocity = row[1:]
                flow[instances == ind] = velocity
 
            points = np.c_[points, np.zeros(points.shape[0])] # Dummy intensity
            points.astype('float32').tofile(to_folder_points + frame_str + ".bin")
            flow.astype('float32').tofile(to_folder_flow + frame_str + ".bin")
        
save_points(0, False)

In [9]:
# Helper to create voxel grid
def get_info(sensor, frame_str, seq_dir):
    points = np.load(seq_dir + "velodyne" + str(sensor) + "/" + frame_str + ".npy") # Point cloud
    instances = np.load(seq_dir + "instances" + str(sensor) + "/" + frame_str + ".npy") # Instances per point
    velocities = np.load(seq_dir + "velocities" + str(sensor) + "/" + frame_str + ".npy") # Per-instance velocity
    flow = np.zeros(points.shape, dtype=np.float32) # Actual movement of things
    for row in velocities:
        ind = int(row[0])
        velocity = row[1:]
        flow[instances == ind] = velocity
    label = np.load(seq_dir + "labels" + str(sensor) + "/" + frame_str + ".npy")
    return [points, instances, flow, label]

In [10]:
# Initial transforms (other lidar to ego sensor)
inv_transforms = {} 
for pose_file in os.listdir(seq_dir + "pose0"):
    t_frame = pose_file.split(".")[0]
    frame_str = get_frame_str(t_frame, t_start, t_end)
    if frame_str:
        ego_pose = np.load(seq_dir + "pose0/" + pose_file)
        for sensor in sensors:
            sensor_pose = np.load(seq_dir + "pose" + str(sensor) + "/" + pose_file)
            inv_sensor = np.linalg.inv(sensor_pose)
            inv_transforms[sensor] = np.matmul(ego_pose, inv_sensor)

In [11]:
def add_point(point, label, grid):
    voxel = np.floor((point + grid_lengths) / voxel_resolution)
    coords = [int(voxel[0]), int(voxel[1]), int(voxel[2]), int(label)]
    if np.all(np.array(coords) < np.array(grid_dim)):
        grid[coords[0], coords[1], coords[2], coords[3]] += 1
    return grid

# Vectorized
def add_points(points, labels, grid):
    # All voxels
    voxels = np.floor((points + grid_lengths) / voxel_resolution).astype(int)
    voxels = np.hstack((voxels, np.reshape(labels, (-1, 1))))
    # Voxels within grid (check max and min)
    valid_voxels = np.all(voxels < np.reshape(np.array(grid_dim), (1, -1)), axis=1) # Max
    valid_voxels = voxels[valid_voxels]
    valid_inds = np.all(valid_voxels >= 0, axis=1) # Min
    valid_voxels = valid_voxels[valid_inds]
    # Convert to list
    valid_coords = [list(valid_voxels[:, 0]), list(valid_voxels[:, 1]), list(valid_voxels[:, 2]), list(valid_voxels[:, 3])]
    
    grid[valid_coords] += 1
    return grid

In [12]:
# Add points along ray to grid
def ray_trace(point, label, grid, sample_spacing):
    vec_norm = np.linalg.norm(point)
    vec_angle = point / vec_norm
    dists = np.arange(vec_norm, 0, -sample_spacing)
    new_points = np.reshape(dists, (-1, 1)) * np.reshape(vec_angle, (1, 3))
    labels = [0] * new_points.shape[0]
    # End Point is label, free points all 0
    labels[0] = label
    grid = add_points(new_points, labels, grid)
    return grid

In [13]:
# Whether any measurements were found
if not os.path.exists(save_dir + "evaluation"):
    os.mkdir(save_dir + "evaluation")
    

# Create voxel grid
for pose_file in os.listdir(seq_dir + "pose0"):
    t_frame = pose_file.split(".")[0]
    frame_str = get_frame_str(t_frame, t_start, t_end)
    voxel_grid = np.zeros((grid_dim))
    if frame_str:
        for sensor in sensors:
            [points, instances, flow, label] = get_info(sensor, t_frame, seq_dir) # Get info for sensor at frame
            transformed_points = np.matmul(points, inv_transforms[sensor][:3, :3]) # Convert points to ego frame
            transformed_points = transformed_points + inv_transforms[sensor][3, :3]
            for i in range(transformed_points.shape[0]):
                voxel_grid = ray_trace(transformed_points[i, :], label[i], voxel_grid, voxel_resolution)
        # Save
        valid_cells = np.sum(voxel_grid, axis=3) > 0
        labels = np.argmax(voxel_grid, axis=3)
        values, counts = np.unique(labels[valid_cells], return_counts=True)
        print(frame_str, values, counts)
        valid_x = xv[valid_cells]
        valid_y = yv[valid_cells]
        valid_z = zv[valid_cells]
        valid_points = np.stack((valid_x, valid_y, valid_z)).T
        valid_labels = labels[valid_cells]
        valid_points.astype('float32').tofile(save_dir + "/evaluation/" + frame_str + ".bin")
        valid_labels.astype('uint32').tofile(save_dir + "/evaluation/" + frame_str + ".label")

/tmp/ipykernel_1227660/2373061310.py:21: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  grid[valid_coords] += 1


000224 [ 0  1  3  4  5  6  7  8  9 10 12 14 19 20] [2989980    5065     361      17     271      26     469     377    1522
     396      59       5      65      59]
000049 [ 0  1  4  5  6  7  8  9 10 12 14 19 20] [3135068   10171      58      67      45     524     275    1332     169
      18      12     108       8]
000085 [ 0  1  4  5  6  7  8  9 10 12 14 19 20] [3147586    9865      20      90      36     398     330    1170     105
      27       5      92      29]
000177 [ 0  1  3  4  5  6  7  8  9 10 12 14 19 20] [2972179    7965     115      15     208      29     330     286    1307
     315      31      15      32      34]
000341 [ 0  1  4  5  6  7  8  9 10 12 19 20 22] [3319407    6441      27     222       9     184     330     845     132
      55      90      16       6]
000271 [ 0  1  3  4  5  6  7  8  9 10 12 14 19 20 22] [2961704    6222     477      29     239      29     451     574    1047
     229      30       2      90     102       1]
000032 [ 0  1  4  5  6  7 

000202 [ 0  1  3  4  5  6  7  8  9 10 12 14 19 20] [2961548    6321     252      12     183      24     415     411    1403
     274      53       8      77      94]
000081 [ 0  1  4  5  6  7  8  9 10 12 14 19 20] [3115360   10274      14      88      33     418     306    1190     157
      38      12      89      16]
000237 [ 0  1  3  4  5  6  7  8  9 10 12 14 19 20] [3030703    5393     351      19     193      20     548     492    1482
     350      38       8      64      55]
000083 [ 0  1  4  5  6  7  8  9 10 12 14 19 20] [3132638    9735      19     113      16     363     311    1228     111
      37      13      92      22]
000345 [ 0  1  4  5  6  7  8  9 10 12 19 20 22] [3319475    7578      11     195      10     187     314    1099     124
      50     112      20      11]
000092 [ 0  1  4  5  6  7  8  9 10 12 14 19 20] [3170255    9214      28     176      18     481     343    1299     132
      38      19     115      21]
000273 [ 0  1  3  4  5  6  7  8  9 10 12 14 19 2

000025 [ 0  1  4  5  6  7  8  9 10 12 14 19 20] [3139165   10092      34      51      17     315     259    1741      70
      13      21     127       8]
000188 [ 0  1  3  4  5  6  7  8  9 10 12 14 19 20] [2931836    8152     142      14     248      45     455     218    1242
     284      33      23      41      66]
000281 [ 0  1  3  4  5  6  7  8  9 10 12 14 19 20 22] [2945163    7308     435      19     326      43     538     439     763
     164      61       5      76      85       1]
000074 [ 0  1  4  5  6  7  8  9 10 12 14 19 20] [3093855   10238      10      61      34     419     248    1136     180
      18       8      62       5]
000118 [ 0  1  4  5  6  7  8  9 10 12 14 19 20] [3157157    9677      25     170      26     505     325    1142     147
      24      33      54      15]
000169 [ 0  1  3  4  5  6  7  8  9 10 12 14 19 20] [3009275    8589      41      11     172      19     414     271    1254
     203      25       7      30      28]
000125 [ 0  1  3  4  5  6 

000134 [ 0  1  4  5  6  7  8  9 10 12 14 19 20] [3138582   10029      25     165       9     365     279    1084     163
      15      21      43      25]
000232 [ 0  1  3  4  5  6  7  8  9 10 12 19 20] [3040534    5066     373      27     142      44     668     345    1524
     395      33      63      81]
000159 [ 0  1  3  4  5  6  7  8  9 10 12 14 19 20] [3086432    8909      42      14     160      22     381     286    1275
     186      34       7      36      18]
000300 [ 0  1  3  4  5  6  7  8  9 10 12 14 19] [3325186    6656     149       4      98      25     326     311     305
     117      28      30      50]
000257 [ 0  1  3  4  5  6  7  8  9 10 12 14 19 20] [3001237    5638     454      31     234      35     425     405    1319
     388      50       1      63      37]
000258 [ 0  1  3  4  5  6  7  8  9 10 12 14 19 20] [2984124    6422     475      29     210      19     453     469    1280
     370      47       1      49      53]
000248 [ 0  1  3  4  5  6  7  8  9 10

000179 [ 0  1  3  4  5  6  7  8  9 10 12 14 19 20] [2969361    8016     113      13     190      20     337     258    1319
     281      35       1      33      33]
000333 [ 0  1  4  5  6  7  8  9 10 12 19 20 22] [3318149    6017      13     226      10     248     283     823     141
      48      81      18       8]
000201 [ 0  1  3  4  5  6  7  8  9 10 12 14 19 20] [2976430    6359     218      11     285      17     373     397    1454
     266      33       4      81      88]
000218 [ 0  1  3  4  5  6  7  8  9 10 12 14 19 20] [2989505    5578     272      17     211      26     529     401    1539
     361      45       7      54      50]
000185 [ 0  1  3  4  5  6  7  8  9 10 12 14 19 20] [2952012    8306     124       9     242      46     499     231    1255
     276      27      22      20      40]
000113 [ 0  1  4  5  6  7  8  9 10 12 14 19 20] [3193149    9701      15     155      36     500     290    1198     152
      30      21      72      12]
000269 [ 0  1  3  4  5  6 

000288 [ 0  1  3  4  5  6  7  8  9 10 12 14 19 20] [2974774    6925     384      25     310      30     562     559     750
     212      34      10     104      61]
000214 [ 0  1  3  4  5  6  7  8  9 10 12 14 19 20] [2986808    5568     307      16     252      25     503     392    1534
     358      46       1      78      77]
000192 [ 0  1  3  4  5  6  7  8  9 10 12 14 19 20] [2938289    7811     181       9     275      33     434     496    1262
     265      37      12      63     101]
000054 [ 0  1  4  5  6  7  8  9 10 12 14 19 20] [3125552   10507      46      97      42     505     291    1332     168
      23      16      82       9]
000021 [ 0  1  4  5  6  7  8  9 10 12 14 19 20] [3162867    9416      34      70      17     263     254    1692     104
      11      33     140      10]
000098 [ 0  1  4  5  6  7  8  9 10 12 14 19 20] [3170291    9115      32     188      20     353     327    1295     140
      28      20     119      26]
000082 [ 0  1  4  5  6  7  8  9 10 12

000349 [ 0  1  4  5  6  7  8  9 10 12 19 20 22] [3281864    7985      15     213       9     172     323    1007     131
      42     113      20      11]
000126 [ 0  1  3  4  5  6  7  8  9 10 12 14 19 20] [3127484    9930       3      16     197      12     382     285    1059
     156      20      31      53      26]
000242 [ 0  1  3  4  5  6  7  8  9 10 12 14 19 20] [3050374    5165     314      22     194      29     547     338    1610
     354      35       3      57      56]
000171 [ 0  1  3  4  5  6  7  8  9 10 12 14 19 20] [3013257    8395      78      15     225      25     381     245    1295
     219      37      11      23      30]
000012 [ 0  1  4  5  6  7  8  9 10 12 14 19 20] [3154107    9535      16      27       8     237     266    1509     117
       9      27     160       9]
000165 [ 0  1  3  4  5  6  7  8  9 10 12 14 19 20] [3045632    8993      42      11     177      27     371     280    1316
     200      37       9      26      25]
000254 [ 0  1  3  4  5  6 

In [14]:
LABEL_COLORS = np.array([
    (255, 255, 255), # None (free) 0
    (70, 70, 70),    # Building 1
    (100, 40, 40),   # Fences 2
    (55, 90, 80),    # Other 3
    (220, 20, 60),   # Pedestrian 4
    (153, 153, 153), # Pole 5
    (157, 234, 50),  # RoadLines 6
    (128, 64, 128),  # Road 7
    (244, 35, 232),  # Sidewalk 8
    (107, 142, 35),  # Vegetation 9
    (0, 0, 142),     # Vehicle 10
    (102, 102, 156), # Wall 11
    (220, 220, 0),   # TrafficSign 12
    (70, 130, 180),  # Sky 13
    (81, 0, 81),     # Ground 14
    (150, 100, 100), # Bridge 15
    (230, 150, 140), # RailTrack 16
    (180, 165, 180), # GuardRail 17
    (250, 170, 30),  # TrafficLight 18
    (110, 190, 160), # Static 19
    (170, 120, 50),  # Dynamic 20
    (45, 60, 150),   # Water 21
    (145, 170, 100), # Terrain 22
]) / 255.0 # normalize each channel [0-1] since is what Open3D uses